TODO: change to use own model instead of pretrained

In [3]:
from pyannote.audio import Pipeline
import os
from dotenv import load_dotenv
import re
from pydub import AudioSegment

load_dotenv()

huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

# TODO: change to own model
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=huggingface_token)

/home/vicuser/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/vicuser/.local/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [1]:
# TODO: niet hardgecodeerd voor eindresultaat
path = "../training_database/wavs/file17.wav"

In [4]:
# apply pretrained pipeline
diarization = pipeline(path)    

/home/vicuser/.local/lib/python3.10/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


: 

create rttm file based on model result

In [4]:
file_name = re.search(r'.*\/(.+?)\.wav', path).group(1)

f = open(f"./results/rttm/{file_name}.rttm", "w")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    f.write(f"SPEAKER {file_name} 1 {round(turn.start, 3)} {round(turn.end-turn.start, 3)} <NA> <NA> {speaker} <NA> <NA>\n")

f.close()

turn result into audio files

In [6]:
audio = AudioSegment.from_wav(f"../training_database/wavs/{file_name}.wav")
os.mkdir(f"./results/audio/{file_name}/")
for turn, _, speaker in diarization.itertracks(yield_label=True):
    if not os.path.isdir(f"./results/audio/{file_name}/{speaker}"):
        os.mkdir(f"./results/audio/{file_name}/{speaker}")

    segment = audio[turn.start * 1000:turn.end * 1000]  # Convert seconds to milliseconds
    segment.export(f"./results/audio/{file_name}/{speaker}/speaker_{speaker}_{turn.start:.1f}-{turn.end:.1f}.wav", format="wav")

create audio file based on speaker by combining audio files

In [7]:
speakers = os.listdir(f"./results/audio/{file_name}")

for speaker in speakers:
    files_list = os.listdir(f"./results/audio/{file_name}/{speaker}")

    combined = AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[0]}")
    for i in range(1,len(files_list)):
        combined = combined + AudioSegment.from_file(f"./results/audio/{file_name}/{speaker}/{files_list[i]}")

    combined.export(f"./results/audio/{file_name}_{speaker}.wav", format="wav")